In [26]:
import pandas as pd
import numpy as np
books = pd.read_csv("./data/books_with_categories.csv")


In [9]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None, device=0)
classifier("I love this!")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8642535e-a649-423d-9b6d-f60902895720)')' thrown while requesting HEAD https://huggingface.co/j-hartmann/emotion-english-distilroberta-base/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764594301581383},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.00041385157965123653}]]

In [14]:
classifier(books["description"][0])


[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.16985194385051727},
  {'label': 'sadness', 'score': 0.11640876531600952},
  {'label': 'surprise', 'score': 0.02070068195462227},
  {'label': 'disgust', 'score': 0.019100716337561607},
  {'label': 'joy', 'score': 0.0151612414047122},
  {'label': 'anger', 'score': 0.003935152664780617}]]

In [15]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296032905578613},
  {'label': 'neutral', 'score': 0.14038528501987457},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.04794234782457352},
  {'label': 'anger', 'score': 0.00915635097771883},
  {'label': 'disgust', 'score': 0.002628469606861472},
  {'label': 'sadness', 'score': 0.002122158883139491}],
 [{'label': 'neutral', 'score': 0.449370801448822},
  {'label': 'disgust', 'score': 0.27359163761138916},
  {'label': 'joy', 'score': 0.10908281058073044},
  {'label': 'sadness', 'score': 0.0936271920800209},
  {'label': 'anger', 'score': 0.04047831892967224},
  {'label': 'surprise', 'score': 0.02697017416357994},
  {'label': 'fear', 'score': 0.006879049353301525}],
 [{'label': 'neutral', 'score': 0.6462163925170898},
  {'label': 'sadness', 'score': 0.24273289740085602},
  {'label': 'disgust', 'score': 0.043422624468803406},
  {'label': 'surprise', 'score': 0.028300542384386063},
  {'label': 'joy', 'score': 0.01421148143

In [22]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [17]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [23]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296032905578613},
 {'label': 'neutral', 'score': 0.14038528501987457},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794234782457352},
 {'label': 'anger', 'score': 0.00915635097771883},
 {'label': 'disgust', 'score': 0.002628469606861472},
 {'label': 'sadness', 'score': 0.002122158883139491}]

In [ ]:
sentences[4]


' John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers'

In [24]:
predictions[4]

[{'label': 'sadness', 'score': 0.9671574234962463},
 {'label': 'neutral', 'score': 0.015104176476597786},
 {'label': 'disgust', 'score': 0.0064806039445102215},
 {'label': 'fear', 'score': 0.005394001957029104},
 {'label': 'surprise', 'score': 0.0022869459353387356},
 {'label': 'anger', 'score': 0.0018428926123306155},
 {'label': 'joy', 'score': 0.0017338803736492991}]

In [25]:
sorted(predictions[0], key=lambda x: x['score'])

[{'label': 'sadness', 'score': 0.002122158883139491},
 {'label': 'disgust', 'score': 0.002628469606861472},
 {'label': 'anger', 'score': 0.00915635097771883},
 {'label': 'joy', 'score': 0.04794234782457352},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'neutral', 'score': 0.14038528501987457},
 {'label': 'surprise', 'score': 0.7296032905578613}]

In [27]:
emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_score = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction, key=lambda x: x['score'], reverse=True)
        for index, label in enumerate(emotion_labels):
            per_emotion_score[label].append(sorted_prediction[index]['score'])
    return {label: np.max(scores) for label, scores in per_emotion_score.items()}
